In [1]:
import torch
from transformers import BertModel, BertTokenizer

ModuleNotFoundError: ignored

In [2]:
!pip install torch

In [3]:
!pip install transformers

     |████████████████████████████████| 1.5MB 6.8MB/s 
     |████████████████████████████████| 2.9MB 22.2MB/s 
     |████████████████████████████████| 890kB 45.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=619af7299c25300b5b55468be7a0917322b431254f2a0db02907b572a5938f68
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch
from transformers import BertModel, BertTokenizer

In [5]:
# input text (string) needs to be converted into some data type (numbers) that the model can use

# Initialize the tokenizer with a pretrained model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
# Convert the string "granola bars" to tokenized vocabulary IDs
granola_ids = tokenizer.encode('granola bars')
# Print the IDs
print('granola_ids', granola_ids)
print('type of granola_ids', type(granola_ids))
# Convert the IDs to the actual vocabulary item
# Notice how the subword unit (suffix) starts with "##" to indicate 
# that it is part of the previous string
print('granola_tokens', tokenizer.convert_ids_to_tokens(granola_ids))


granola_ids [101, 12604, 6030, 6963, 102]
type of granola_ids <class 'list'>
granola_tokens ['[CLS]', 'gran', '##ola', 'bars', '[SEP]']


In [7]:
# Convert the list of IDs to a tensor of IDs 
granola_ids = torch.LongTensor(granola_ids)
# Print the IDs
print('granola_ids', granola_ids)
print('type of granola_ids', type(granola_ids))

granola_ids tensor([  101, 12604,  6030,  6963,   102])
type of granola_ids <class 'torch.Tensor'>


In [8]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
# Set the device to GPU (cuda) if available, otherwise stick with CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)
granola_ids = granola_ids.to(device)

model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [9]:
print(granola_ids.size())
# unsqueeze IDs to get batch size of 1 as added dimension
granola_ids = granola_ids.unsqueeze(0)
print(granola_ids.size())

print(type(granola_ids))
with torch.no_grad():
    out = model(input_ids=granola_ids)

# the output is a tuple
print(type(out))
# the tuple contains three elements as explained above)
print(len(out))
# we only want the hidden_states
hidden_states = out[2]
print(len(hidden_states))

torch.Size([5])
torch.Size([1, 5])
<class 'torch.Tensor'>
<class 'transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions'>
3
13


In [11]:
sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze()
print(sentence_embedding)
print(sentence_embedding.size())

tensor([ 2.7497e-01,  1.8313e-01, -8.8652e-02,  2.1698e-01,  3.1942e-01,
        -1.1412e-01,  7.4039e-02,  3.7655e-01, -4.1821e-01,  9.9971e-02,
        -9.0241e-02, -2.4298e-01,  1.5542e-01,  4.2042e-01, -2.5547e-01,
         2.9753e-01, -2.9643e-01, -2.5810e-02,  8.5306e-02,  1.0182e-01,
         3.0401e-01, -4.4263e-01,  3.1249e-02,  1.4435e-01,  3.0189e-01,
         7.3914e-02, -2.5580e-01,  3.1384e-01, -1.4688e-01, -1.5202e-01,
         7.0785e-02,  4.0448e-01, -1.1769e-01,  3.1848e-01,  2.8022e-02,
        -1.6934e-01,  3.5639e-01, -2.2931e-01, -1.1899e-01, -1.1182e-01,
        -1.6003e-01,  7.9355e-02,  5.1107e-01,  5.2223e-02, -1.5481e-01,
         2.8228e-02, -1.4365e-01, -4.7737e-01, -5.6638e-01, -4.8802e-01,
        -1.1429e-01,  2.8087e-01, -5.7160e-02,  2.3862e-01,  3.5440e-01,
         5.8237e-01,  1.2777e-01,  1.0363e-01,  3.0538e-01,  2.0989e-01,
         1.1693e-01,  2.6346e-01, -1.5832e-01, -1.1380e-01,  1.7190e-02,
        -3.4662e-02,  1.1470e-01,  3.2023e-02, -1.9

In [12]:
cleaned_100 = ['BRAKE PEDAL PUSH ROD RETAINER WAS NOT PROPERLY INSTALLED, CAUSING BRAKES TO FAIL, RESULTING AN ACCIDENT AFTER RECALL REPAIRS (94V-129).', 'VEHICLE STALLS AT HIGH SPEED, RESULTING LOSS OF STEERING AND BRAKING ABILITY.', 'EXHAUST SYSTEM FAILS; PLEASE DESCRIBE DETAILS.', 'BRAKING SYSTEM FAILURE WITHOUT ABS BRAKES.', 'VEHICLES SUN ROOF GLASS FLEW OFF WHILE DRIVING.', "WHEN REAR ENDED WITH FOOT BRAKE, DRIVERS SHOULDER BELT DIDN'T RESTRAIN FROM MAKING CONTACT, RESULTING AN INJURY. ESTIMATED SPEED OF IMPACT 30 MPH.", 'VEHICLE STALLS AT HIGH SPEED, RESULTING LOSS OF STEERING AND BRAKING ABILITY.', 'VEHICLE STALLS AT HIGH SPEED, RESULTING LOSS OF STEERING AND BRAKING ABILITY.', 'PARKED FLAT SURFACE EMERGENCY BRAKING ENGAGED VEHICLE ROLLED REARWARD.', 'ABS SYSTEM FAILURE, AT 20MPH.', 'UPON FRONTAL COLLISION, AIR BAG FAILED TO DEPLOY. VEHICLE CLASSIFIED AS TOTALED. PLEASE DESCRIBE DETAILS.', 'FRONT TIRES SHOW EXCESSIVE WEAR AND STEEL BELTS. NISSAN IS OFFERING TO REPLACE TWO TIRES.', 'THE STRUT WAS BAD THERE IS NOISE THE PASSENGER SIDE DOOR AND THE ENGINE LIGHT MALFUNCTION.', 'VEHICLE SUDDENLY ACCELERATED OUT OF CONTROL, BRAKE DID NOT RESPOND, RESULTING ACCIDENT.', 'THROTTLE BODY STUCK OPEN POSITION, CAUSING VEHICLE TO ACCELERATE OUT OF CONTROL.', 'DRIVER AIR BAG DID NOT DEPLOY DURING FRONTAL COLLISON.', 'REAR MIDDLE SEAT BELT RETRACTOR DOES NOT OPERATE.', 'THE STRUT WAS BAD THERE IS NOISE THE PASSENGER SIDE DOOR AND THE ENGINE LIGHT MALFUNCTION.', 'REPLACED TRANSMISSION DUE TO INTERNAL FLUID LEAK THE LOW REVERSE CLUTCH CIRCUIT.', 'ERRATIC OPERATION OF ELECTRONIC GAS GAUGE. UPON STARTING GALLON OF GAS POSSIBLY IS LOST.', 'AUTOMATIC DOOR LOCKS WHEN USED, WILL NOT RELEASE FROM ANY OF THE FOUR DOORS WHEN ENGINE IS TURNED OFF.', 'HEADLIGHTS WILL COME AND OFF INTERMITTENTLY.', 'VEHICLE SUFFERED SUB-FRAME ABSORBER FAILURE.(PE94-048).', 'ENGINE MOTOR MOUNTS FAILED, RESULTING ENGINE NOISE.', 'ABS BRAKES FAILS TO STOP VEHICLE EFFECTIVELY WHEN APPLIED CAUSING EXTENDED STOPPING. FRONTEND COLLISION AT 45-50MPH. NO INJURIES. PLEASE DESCRIBE DETAILS.', 'IT IS HARD TO SHIFT GEARS, SHIFTS ARE MISSED BECAUSE TRANSMISSION WILL NOT ENGAGE REVERSE/1ST/2ND/3RD/4TH GEARS.', 'THROTTLE STICKS WHICH CAUSED AN ACCIDENT.', 'DRIVER SEAT BELT RETRACTOR STOPS WORKING WHEN DOOR IS CLOSED CAN HEAR MOTOR CLIP AND GO BACK HALF WAY.', 'DRIVER SIDE SEAT FRAME BROKE TWO, CAUSING SEAT NOT TO BE ABLE REMAIN AN UPRIGHT POSITION.', 'DURING BRAKE APPLICATION VEHICLE EXHIBITS GRINDING NOISE.', 'MODEL #690. THE HARD PLASTIC BOTH SIDES ARE CRACKING PURCHASE BRAND NEW.', 'STRESS CRACKS AROUND FRONT PLASTIC HINGE.', 'HORN PAD SHORTED OUT.', 'SEAT BELT FAILED ABOVE ACCIDENT, RESULTING AN INJURY.', 'STEERING WHEEL BOLTS LOOSENEDAND ROCKED BACK AND FORTH, RUINING THE THREADS AND CAUSING BOLTS TO FALL OUT, MAKING IT DIFFICULT TO TURN WHEEL. AK', 'REAR BEAM SUPPORT BY THE FRAME CRACKED, RESULTING CONTROL LOST AT 10MPH.', 'WINDSHIELD IS PITTED, CAUSING GLARE SUNLIGHT.', "THE SLIDING DOOR FALL OFF THE HINGE FOR THE SECOND TIME AND THE WINDSHEILD WIPER THE REAR DOESN'T WORK AND THE DOOR HANDLE SIDE FELL OFF.", "THE SLIDING DOOR FALL OFF THE HINGE FOR THE SECOND TIME AND THE WINDSHEILD WIPER THE REAR DOESN'T WORK AND THE DOOR HANDLE SIDE FELL OFF.", 'REPLACED LEAKING MASTER CYLINDER.', 'OTHER PERSON INJURIED. FRAME SYSTEM DESIGNED TO RECIEVE SNOW PLOW, FRAME ACTS AS CAN OPENER UPON IMPACT.', 'PASSENGER AIR BAG DID NOT DEPLOY DURING FRONTAL COLLISON.', 'DURING REAR SIDE-IMPACT COLLISION AT 15-20PH, REAR TAILGATE LATCH FAILED, CAUSING TAILGATE DOOR TO FLY OPEN AND BELONGINGS INSIDE OF VEHICLE FELL OUT.', 'CAR STALLS FOR NO APPARENT REASON.', 'BRAKES FAILED DURING SUDDEN ACCELERATION.', 'INTERMITTENTLY OWNER HAS PROBLEMS STARTING, VEHICLE HAS DIGITAL DISPLAY WITH EVERYTHING NEEDING TO BE RESET.', 'DRIVERS LIGHTS FLICKER OFF WHEN USED.', 'RIGHT FRONT WHEEL LOCKED UP VEHICLE WENT INTO SPIN. NO RESPONSE FROM ANTI LOCK BRAKES.', "CAR WAS AN ACCIDENT AND AIR BAG DIDN'T DEPLOY THE SPEED LIMIT WAS 30 MPH.", 'CAR IS STALLING, FOR NO APPARENT REASON.', 'MISALIGNMENT, CAUSING VEHICLE TO PULL TO THE RIGHT.', 'IDLE SPEED CONTROL UNIT FAILED, CAUSING VEHICLE TO STALL.', 'VEHICLE ROLLED DOWN HILL WHILE PARK, AND INTO TREE.', 'BRAKES WOULD NOT WORK SEVERAL OCCASIONS, LAST INSTANCE RESULTED AN ACCIDENT.', 'DUE TO DEFECT STEERING BOLTS, STEERING WHEEL IS TOO LOOSE FOR PROPER CONTROL WHILE DRIVING. PLEASE DESCRIBE.', 'BRAKE FAILURE EQUIPPED WITH ABS BRAKES.', 'EVENFLO MODEL #225 CHILD SEAT BUCKLE TEE SHIELD STICKS, MAKING IT DIFFICULT TO UNLATCH.', 'BREAKAGE OF BALL JOINTS AT LOW RATE SPEEDS. CAUSING FRONT WHEEL TO SLIDE OUT TO THE SIDE.', 'VEHICLES DRIVER SEAT MOVES AND IS UNADJUSTABLE.', 'LEAKING GAS TANK UNDER REAR PLATFORM FLOOR.', 'FUEL LEAKED FROM FUEL TANK AREA, EMITTING STRONG FUMES GARAGE.', 'SHIFTED INTO REVERSE VEHICLE JERKED VIOLENTLY. RESULTING VEHICLE ACCIDENT.', 'FUEL TANK ; LEAKS BECAUSE OF RUST GAS LEAK BY THE EXHAUST SYSTEM.', 'AIR BAG; THREE TIMES DEALERSHIP FOR SRS SIGNAL CONTINUOUSLY ILLUMINATING. ZONE REP ALSO CAME TO INVESTIGATE WHY LIGHT STAYS PLEASE DESCRIBE DETAILS.', 'REPLACED FRONT BRAKE ROTORS/PADS.', 'POWER DOOR LOCK PASSENGER SIDE FAILED TO OPEN, WHEN LOCK WAS DISENGAGED.', 'WHILE DRIVING EXPERIENCES POWER STEERING FAILURE, RENDERING STEERING CAPABILITY INOPERATIVE.', 'EGR VALVE FAILED, CAUSING VEHICLE TO STALL.', 'UPON IMPACT, DURING FRONTAL CRASH AT APPROX. 10 MPH, 2:00 FRONT BUMPER, DRIVERS/PASSENGERS AIRBAGS FAILED TO DEPLOY, RESULTING INJURY.', 'SEAT ADJUSTMENT BAR BROKE.', 'ABS LIGHT CAME BEFORE MASTER CYLINDER REPLACE, KNOW IT IS NOT BEFORE MALFUNCTION, ABS BRAKE SYSTEM LOCKUP DURING ICE WEATHER, REPLACE MASTER CYLINDER.', 'REAR DOOR LATCH DOES NOT LOCK, NO PART REPLACED.', "THE SLIDING DOOR FALL OFF THE HINGE FOR THE SECOND TIME AND THE WINDSHEILD WIPER THE REAR DOESN'T WORK AND THE DOOR HANDLE SIDE FELL OFF.", 'ABS BRAKE SYSTEM MALFUNCTIONED, VERY LITTLE PRESSURE LEFT THE SYSTEM, RESULING EXTENDED STOPPING DISTANCE, REPLACED ABS SYSTEM.', 'GAS TANK MOUNTED OUT SIDE THE FRAME RAIL.', 'STEERING LOCKED UP WHILE DRIVING, REPLACED RACK AND PUMP, TO NO AVAIL.', "CAR WAS AN ACCIDENT AND THE DRIVER SIDE AUTOMATIC SEAT BELT DIDN'T RESTRAIN, RESULTING AN INJURY, CONSUMER DOESN'T HAVE THIS CAR ANYMORE, HAS NEW CAR.", 'WHEN DRIVING CAME TO STOP SIGN CAR ACCELERATED AND WHEN STARTED CAR IT HESITATED CAR BEEN TO DEALER ABOUT FIVE TIMES WITH THIS PROBLEM NOT BEING FOUND.', 'CABLE ATTACHMENT THAT SECURES THE SPARE TIRE BROKE WHILE DRIVING.', 'DRIVER/REAR SEATBELTS FASTNERS FAILS TO STAY LATCHED. PLEASE DESCRIBE DETAILS.', 'DURING IMPACT, DRIVER AIR BAG CAME OUT BUT DID NOT FULLY INFLATE.', 'BATTERY MALFUNCTIONED WHEN TOO MUCH POWER WAS DRAWN FROM BATTERY FOR RADIO.', 'BRAKES FAILED DUE TO BATTERY MALFUNCTIONING WHEN TOO MUCH POWER WAS DRAWN FROM BATTERY FOR RADIO.', 'STEERING FAILED DUE TO BATTERY MALFUNCTIONING BECAUSE TOO MUCH POWER WAS DRAWN FROM BATTERY FOR RADI.', 'WHILE DRIVING, NOTICED SMOKE COMING FROM UNDER HOOD, THEN ENGINE CAUGHT FIRE, FIRE DEPT SUSPECTS ELECTRIAL SYSTEM AS CULPRIT.', 'ABS BRAKING FAILURE, AT 35-45 MPH.', 'CRUISE CONTROL MALFUNCTIONED TWICE TWO MONTHS.', 'RADIATOR FAILED @ HIGHWAY SPEED OBSTRUCTING DRIVERS VISION TEMPORARY. PLEASE DESCRIBE DETAILS.', 'DRIVER SIDE POWER SEAT CAUGHT FIRE.', 'TRANSMISSION NOT SHIFTING OUT OF 2ND GEAR, WENT INTO LIM MODE/TRANSMMION NOT SHIFTING THEN JUMPED OUT OF GEAR.', 'ABS, BRAKE FAILURE WHILE DRIVING WENT OVER BUMP LOST BRAKE EFFECTIVENESS EA94-038.', 'TRANSMISSION; PURCHASED VAN BRAND NEW AND TRANSMISSION BLEW.', "ABS BRAKES; TRUCK DIDN'T RESPOND TO PEDAL TRAVEL. THIS HAS OCCURED FIVE TIMES.", 'RADIATOR LEAKED, EVERYTHING LOCKED UP, WHEN DRIVING IT WAS HARD TURN THE STEERING WHEEL.', 'PULLEY BROKE OFF, EVERYTHING LOCKED UP, WHEN DRIVING HARD TO TURN THE STEEERING WHEEL.', 'PROBLEM WITH RENOLUX GT4000 DATED JUNE 1993 CHILD SAFETY SEAT PRIOR TO SERVICE RECALL.', "WAS AN ACCIDENT AND THE AIR BAG DIDN'T DEPLOY.", 'IGNITION SWITCH FAILURE, CAUSING INTERMITTENT CRANCKING/STARTING PROBLEMS.', 'RIGHT FRONT WHEEL LOCKED UP VEHICLE WENT INTO SPIN. NO RESPONSE FROM ANTI LOCK BRAKES.', 'ABS: EXPERIENCED ANTI-LOCK BRAKE FAILURE/EXTENDED STOPPING DISTANCE, RESULTING AN ACCIDENT, IMPACT 12:00 POSITION SPEED, 20 MPH.']

In [14]:
print(cleaned_100[1])

VEHICLE STALLS AT HIGH SPEED, RESULTING LOSS OF STEERING AND BRAKING ABILITY.


In [28]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

embeddings = []

for i in range(100):
  sent = cleaned_100[i]
  #print(sent)
  ids = tokenizer.encode(sent)
  #print(ids)
  ids = torch.LongTensor(ids)
  ids = ids.to(device)
  model.eval()
  ids = ids.unsqueeze(0)
  #print(ids)
  with torch.no_grad():
    out = model(input_ids=ids)
  hidden_states = out[2]
  sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze()
  #print(sentence_embedding)
  embeddings.append(sentence_embedding)

In [29]:
# the above script took 18 seconds to run

In [30]:
print(embeddings[0])

tensor([-2.5874e-01, -3.5105e-01,  2.6384e-01,  1.4098e-01,  4.7970e-01,
         3.0390e-02,  6.9183e-01, -1.4386e-01, -9.4627e-02,  1.0402e-01,
         2.1764e-01, -1.2120e-01,  3.2090e-02, -1.4672e-01, -3.3392e-01,
         1.7578e-01,  3.1360e-01, -2.5112e-01, -3.5943e-02,  2.4619e-01,
         1.9450e-01,  1.9478e-01, -6.1660e-02,  5.6187e-01,  1.2839e-01,
         2.3944e-01, -2.9448e-01,  2.2979e-01, -3.3562e-01, -1.7624e-01,
         3.6844e-01,  4.1380e-03, -3.0246e-01, -4.2393e-02,  2.1231e-01,
        -1.1511e-01, -8.9833e-02, -4.0605e-01, -3.5504e-02,  3.5247e-01,
        -6.0220e-01, -4.0937e-01, -1.6859e-01, -1.6765e-01,  2.9482e-01,
         3.2267e-01,  1.5531e-02,  1.3759e-01,  4.3259e-01, -1.1664e-01,
        -5.0936e-01,  1.6482e-01,  3.7377e-03, -1.6615e-01,  1.1936e-01,
         8.0688e-01, -3.1774e-01, -7.8410e-01,  6.3453e-02,  1.8069e-02,
         5.9232e-02, -1.3546e-01, -1.7084e-01, -5.3157e-02,  1.2622e-01,
         8.8040e-02,  7.3258e-02,  7.4271e-01, -3.2

In [31]:
print(embeddings[1])

tensor([-4.9514e-01, -9.6689e-02,  2.1033e-01, -7.0137e-02,  2.6487e-01,
        -1.2907e-01,  4.5103e-01,  5.2163e-01,  1.9886e-01, -1.8036e-01,
        -2.4982e-01, -2.6748e-01,  1.7252e-01,  8.1478e-02, -4.4773e-01,
         3.4286e-01,  3.8899e-01,  1.7072e-01, -3.5713e-01,  4.4280e-01,
         4.2338e-03, -1.6486e-01, -3.3657e-01,  3.8811e-01,  4.7346e-01,
        -3.8388e-02, -1.9978e-01,  4.1737e-01, -3.2785e-01, -5.7204e-01,
         2.5291e-01,  1.5715e-01, -2.1334e-01,  6.2125e-01,  5.0203e-01,
        -1.2200e-01, -8.2576e-02, -1.9409e-01, -1.7714e-01,  2.3262e-01,
        -6.5334e-01, -2.6660e-01, -5.1914e-02,  1.5588e-02,  2.2195e-01,
         1.9890e-02,  3.0815e-01,  3.4847e-02,  5.7808e-01, -2.0489e-01,
        -3.1153e-01,  2.1891e-01, -1.2339e-01, -2.3691e-02,  2.5615e-01,
         7.8680e-01, -4.9394e-01, -8.0745e-01,  3.6406e-01, -7.7552e-03,
         2.8208e-01, -3.1431e-01, -2.5558e-01, -1.8495e-01,  4.0340e-01,
        -9.5840e-02, -6.2958e-03,  9.5195e-01, -8.3

In [32]:
for i in range(len(embeddings[0])):
  print(embeddings[0][i] - embeddings[1][i])

tensor(0.2364)
tensor(-0.2544)
tensor(0.0535)
tensor(0.2111)
tensor(0.2148)
tensor(0.1595)
tensor(0.2408)
tensor(-0.6655)
tensor(-0.2935)
tensor(0.2844)
tensor(0.4675)
tensor(0.1463)
tensor(-0.1404)
tensor(-0.2282)
tensor(0.1138)
tensor(-0.1671)
tensor(-0.0754)
tensor(-0.4218)
tensor(0.3212)
tensor(-0.1966)
tensor(0.1903)
tensor(0.3596)
tensor(0.2749)
tensor(0.1738)
tensor(-0.3451)
tensor(0.2778)
tensor(-0.0947)
tensor(-0.1876)
tensor(-0.0078)
tensor(0.3958)
tensor(0.1155)
tensor(-0.1530)
tensor(-0.0891)
tensor(-0.6636)
tensor(-0.2897)
tensor(0.0069)
tensor(-0.0073)
tensor(-0.2120)
tensor(0.1416)
tensor(0.1198)
tensor(0.0511)
tensor(-0.1428)
tensor(-0.1167)
tensor(-0.1832)
tensor(0.0729)
tensor(0.3028)
tensor(-0.2926)
tensor(0.1027)
tensor(-0.1455)
tensor(0.0882)
tensor(-0.1978)
tensor(-0.0541)
tensor(0.1271)
tensor(-0.1425)
tensor(-0.1368)
tensor(0.0201)
tensor(0.1762)
tensor(0.0234)
tensor(-0.3006)
tensor(0.0258)
tensor(-0.2229)
tensor(0.1788)
tensor(0.0847)
tensor(0.1318)
tensor(-0.